In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.genmod.families.links import Log
from sklearn.metrics import mean_squared_error

def load_data(log_scaled=False):
    url = "https://raw.githubusercontent.com/jh85/glm/refs/heads/main/used_cars_price.csv"
    df = pd.read_csv(url)

    drop_cols = ["car_name", "car_make", "car_model", "car_spec", "location", "year", "power"]
    df.drop(columns=drop_cols, inplace=True)

    # For "kmpl"
    x = df[df.mileage_unit == "kmpl"].mileage
    x_min = x.min()
    x_max = x.max()
    df["kmpl_temp"] = x.apply(lambda x: (x - x_min / (x_max - x_min)))

    # For "km/kg"
    y = df[df.mileage_unit == "km/kg"].mileage
    y_min = y.min()
    y_max = y.max()
    df["kmkg_temp"] = y.apply(lambda y: (y - y_min / (y_max - y_min)))

    # Combining scaled mileage
    df["mileage_scaled"] = df["kmpl_temp"].combine_first(df["kmkg_temp"])

    # Drop columns "kmpl_temp", "kmkg_temp", "mileage_unit" and "mileage"
    df.drop(["kmpl_temp", "kmkg_temp", "mileage_unit", "mileage"], axis=1, inplace=True)
    
    # Apply log transformation to specified columns
    if log_scaled == True:
        log_cols = ["engine", "kilometers_driven", "price", "mileage_scaled"]
        for col in log_cols:
            # The minimum value in the price column is 0.44, and np.log(0.44) results in a negative value.
            # Since Gamma regression requires a strictly positive response variable, we add a padding value (1.0).
            df[col] = np.log(df[col] + 1.0)

    # Get list of categorical values for onehot encoding
    vars_req_encoding = df.select_dtypes(exclude=np.number).columns.to_list()
    df = pd.get_dummies(df, columns=vars_req_encoding, drop_first=True, dtype=float)

    X = df.drop(["price"], axis=1)
    y = df[["price"]]

    return X, y

def main():
    # Construct and fit a Gamma Regression model with a log link function
    X,y = load_data(log_scaled=True)
    glm_X = sm.add_constant(X)
    # Note: The default link function for Gamma regression in GLM is InversePower.
    gamma_model = sm.GLM(y,glm_X,family=sm.families.Gamma(link=Log()))
    gamma_result = gamma_model.fit()
    print(gamma_result.summary())
    gamma_preds = gamma_result.predict(exog=glm_X)

    # Construct and fit a Linear Regression model
    X,y = load_data(log_scaled=True)
    glm_X = sm.add_constant(X)
    linear_model = sm.GLM(y,glm_X,family=sm.families.Gaussian())
    linear_result = linear_model.fit()
    print(linear_result.summary())
    linear_preds = linear_result.predict(exog=glm_X)

    gamma_mse = mean_squared_error(y, gamma_preds)
    linear_mse = mean_squared_error(y, linear_preds)
    print(f"MSE: gamma {gamma_mse} linear {linear_mse}")

main()

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  price   No. Observations:                 6016
Model:                            GLM   Df Residuals:                     5997
Model Family:                   Gamma   Df Model:                           18
Link Function:                    Log   Scale:                        0.017215
Method:                          IRLS   Log-Likelihood:                -336.50
Date:                Sun, 19 Jan 2025   Deviance:                       110.51
Time:                        02:41:05   Pearson chi2:                     103.
No. Iterations:                    13   Pseudo R-squ. (CS):             0.9991
Covariance Type:            nonrobust                                         
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                 